Import Required Libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')


from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.linear_model import LinearRegression
from sklearn.cluster import KMeans
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder


Connect to MYSQL

In [ ]:
# load the investment sample dataset
Investment_data = pd.read_csv(r"C:\Users\Hp\Documents\Investment Recommender system\sample_investment_data.csv")
Investment_data

,Investor_ID,Investor_Type,Sector,LGA,Investment_Type,Amount_NGN,Year,ROI_Percent,Status
0,INV000,Local,Mining,Lafia,DDI,444398601,2022,13.30,Stalled
1,INV001,NGO,Manufacturing,Nasarawa,FDI,316835156,2023,7.96,Stalled
2,INV002,Foreign,Energy,Lafia,FDI,282515989,2018,5.75,Active
3,INV003,NGO,Manufacturing,Nasarawa,FDI,194736815,2020,19.10,Completed
4,INV004,Local,Healthcare,Doma,FDI,102974379,2020,14.74,Completed
...,...,...,...,...,...,...,...,...,...
95,INV095,Local,Healthcare,Nasarawa,FDI,333748685,2022,23.13,Stalled
96,INV096,Local,Mining,Akwanga,DDI,394128720,2019,7.83,Stalled
97,INV097,Foreign,Technology,Lafia,FDI,173990418,2020,7.17,Completed
98,INV098,Local,Healthcare,Lafia,FDI,214260227,2024,19.41,Active


In [4]:
Investment_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Investor_ID      100 non-null    object 
 1   Investor_Type    100 non-null    object 
 2   Sector           100 non-null    object 
 3   LGA              100 non-null    object 
 4   Investment_Type  100 non-null    object 
 5   Amount_NGN       100 non-null    int64  
 6   Year             100 non-null    int64  
 7   ROI_Percent      100 non-null    float64
 8   Status           100 non-null    object 
dtypes: float64(1), int64(2), object(6)
memory usage: 7.2+ KB


In [5]:
# View the summary statistics of the Investment dataset
Investment_data.describe()

,Amount_NGN,Year,ROI_Percent
count,1.000000e+02,100.000000,100.000000
mean,2.524294e+08,2020.860000,13.695500
std,1.330669e+08,1.922856,6.003145
min,2.010028e+07,2018.000000,5.110000
25%,1.458326e+08,2019.000000,8.347500
50%,2.419006e+08,2021.000000,13.315000
75%,3.593637e+08,2022.000000,19.150000
max,4.873780e+08,2024.000000,24.560000


In [3]:
# load the investor profile dataset
Investor_profile = pd.read_csv(r"C:\Users\Hp\Documents\Investment Recommender system\sample_investor_profiles.csv")
Investor_profile

,Investor_ID,Country,Risk_Appetite,Preferred_Sector,Budget_Range_NGN,Years_Active
0,INV000,Nigeria,High,Technology,63M–387M,5
1,INV001,India,Medium,Healthcare,82M–209M,5
2,INV002,China,Medium,Technology,94M–246M,9
3,INV003,India,Medium,Technology,10M–264M,3
4,INV004,Nigeria,Medium,Manufacturing,91M–169M,3
...,...,...,...,...,...,...
95,INV095,USA,Low,Mining,73M–413M,10
96,INV096,Germany,Low,Agriculture,40M–407M,7
97,INV097,China,Low,Agriculture,69M–391M,3
98,INV098,UK,High,Mining,71M–389M,1


Check Data Overview

In [6]:
Investor_profile.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Investor_ID       100 non-null    object
 1   Country           100 non-null    object
 2   Risk_Appetite     100 non-null    object
 3   Preferred_Sector  100 non-null    object
 4   Budget_Range_NGN  100 non-null    object
 5   Years_Active      100 non-null    int64 
dtypes: int64(1), object(5)
memory usage: 4.8+ KB


In [7]:
Investor_profile.describe()

,Years_Active
count,100.000000
mean,4.980000
std,2.696725
min,1.000000
25%,3.000000
50%,5.000000
75%,7.000000
max,10.000000


In [ ]:
# Merge the datasets on 'Investment ID'
merged_data = pd.merge(Investment_data, Investor_profile, on='Investor_ID')